In [1]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster_kwargs = {
    "cores": 128,  
    "memory": "64 GB",
    "shebang": "#!/bin/bash",
    "account": "m3828",
    "walltime": "00:10:00",
    "job_mem": "0",
    "job_script_prologue": ["source ~/.bashrc"],
    "job_directives_skip": ["-n", "--cpus-per-task"],  
    "job_extra_directives": ["-q debug", "-C gpu"],  
}

cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())

slurm_jobs = 1
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)

/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33813 instead
  warnings.warn(


#!/bin/bash

#SBATCH -J dask-worker
#SBATCH -A m3828
#SBATCH --mem=0
#SBATCH -t 00:10:00
#SBATCH -q debug
#SBATCH -C gpu
source ~/.bashrc
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.14:40709 --name dummy-name --nthreads 8 --memory-limit 3.73GiB --nworkers 16 --nanny --death-timeout 60



In [8]:
from mlip_arena.models.utils import EXTMLIPEnum

print(EXTMLIPEnum.MACE in EXTMLIPEnum)

True


In [2]:
from typing import Union

import covalent as ct
import numpy as np
import pandas as pd
import torch
from ase import Atoms
from ase.calculators.calculator import Calculator
from ase.data import chemical_symbols
from matplotlib import pyplot as plt
from prefect import flow, task
from prefect_dask import DaskTaskRunner

from mlip_arena.models import MLIPCalculator
from mlip_arena.models.utils import EXTMLIPEnum, MLIPMap, external_ase_calculator

# chemical_symbols.remove("X")


@task
def calculate_single_diatomic(
    calculator_name: str,
    calculator_kwargs: dict | None,
    atom1: str,
    atom2: str,
    rmin: float = 0.1,
    rmax: float = 6.5,
    npts: int = int(1e3),
) -> dict:

    calculator_kwargs = calculator_kwargs or {}

    if calculator_name in EXTMLIPEnum:
        calc = external_ase_calculator(calculator_name, **calculator_kwargs)
    elif calculator_name in MLIPMap:
        calc = MLIPMap[calculator_name](**calculator_kwargs)

    a = 2 * rmax

    rs = np.linspace(rmin, rmax, npts)
    e = np.zeros_like(rs)
    f = np.zeros_like(rs)

    da = atom1 + atom2

    for i, r in enumerate(rs):

        positions = [
            [0, 0, 0],
            [r, 0, 0],
        ]

        # Create the unit cell with two atoms
        atoms = Atoms(da, positions=positions, cell=[a, a, a])

        atoms.calc = calc

        e[i] = atoms.get_potential_energy()
        f[i] = np.inner(np.array([1, 0, 0]), atoms.get_forces()[1])

    return {"r": rs, "E": e, "F": f, "da": da}


from dask.distributed import Client
from dask_jobqueue import SLURMCluster


@flow(
    task_runner=DaskTaskRunner(
        cluster_class=SLURMCluster, cluster_kwargs=cluster_kwargs
    ),
    log_prints=True
)
def calculate_homonuclear_diatomics(calculator_name, calculator_kwargs):


    results = {}

    for atom in chemical_symbols:
        output = calculate_single_diatomic.submit(
            calculator_name, calculator_kwargs, atom, atom
        )
        print(output)
        da = output.pop("da")
        results[da] = output
        

    return results

/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
calculate_homonuclear_diatomics("MACE", {}).result()

23:20:06.274 | INFO    | prefect.engine - Created flow run 'bizarre-mantis' for flow 'calculate-homonuclear-diatomics'

23:20:06.278 | INFO    | Flow run 'bizarre-mantis' - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/flow-runs/flow-run/84d3805b-5f63-41d5-9103-3039688d59f3

23:20:06.279 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `dask_jobqueue.slurm.SLURMCluster`

/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38453 instead
  warnings.warn(


23:20:06.301 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://128.55.64.14:38453/status

...

23:20:08.275 | INFO    | Flow run 'bizarre-mantis' - Submitted task run 'calculate_single_diatomic-118' for execution.

23:20:08.280 | INFO    | Flow run 'bizarre-mantis' - Created task run 'calculate_single_diatomic-110' for task 'calculate_single_diatomic'

23:20:08.284 | INFO    | Flow run 'bizarre-mantis' - Submitted task run 'calculate_single_diatomic-110' for execution.

23:27:44.390 | ERROR   | Flow run 'bizarre-mantis' - Crash detected! Execution was cancelled by the runtime environment.

KeyboardInterrupt: 

In [ ]:
from typing import Union

import covalent as ct
import numpy as np
import pandas as pd
import torch
from ase import Atoms
from ase.calculators.calculator import Calculator
from ase.data import chemical_symbols
from matplotlib import pyplot as plt

from mlip_arena.models import MLIPCalculator
from mlip_arena.models.utils import EXTMLIPEnum, MLIPMap, external_ase_calculator

local = ct.executor.LocalExecutor()


@ct.electron
def calculate_single_diatomic(
    calculator_name: str,
    calculator_kwargs: dict | None,
    atom1: str,
    atom2: str,
    rmin: float = 0.1,
    rmax: float = 6.5,
    npts: int = int(1e3),
):

    calculator_kwargs = calculator_kwargs or {}

    if calculator_name in EXTMLIPEnum:
        calc = external_ase_calculator(calculator_name, **calculator_kwargs)
    elif calculator_name in MLIPMap:
        calc = MLIPMap[calculator_name](**calculator_kwargs)

    a = 2 * rmax

    rs = np.linspace(rmin, rmax, npts)
    e = np.zeros_like(rs)
    f = np.zeros_like(rs)

    da = atom1 + atom2

    for i, r in enumerate(rs):

        positions = [
            [0, 0, 0],
            [r, 0, 0],
        ]

        # Create the unit cell with two atoms
        atoms = Atoms(da, positions=positions, cell=[a, a, a])

        atoms.calc = calc

        e[i] = atoms.get_potential_energy()
        f[i] = np.inner(np.array([1, 0, 0]), atoms.get_forces()[1])

    return rs, e, f, da


@ct.lattice
def calculate_homonuclear_diatomics(
    calculator_name: str, calculator_kwargs: dict | None = None
):

    chemical_symbols.remove("X")

    results = {}

    for atom in chemical_symbols:
        rs, e, f, da = calculate_single_diatomic(
            calculator_name, calculator_kwargs, atom, atom
        )
        results[da] = {"r": rs, "E": e, "F": f}

    return results



dispatch_id = ct.dispatch(calculate_homonuclear_diatomics)(calculator_name="MACE", calculator_kwargs={})

result = ct.get_result(dispatch_id)

In [ ]:
from enum import Enum

e = Enum("TESTEnum", {"A": 1, "B": 2, "C": 3})

print(e.A)
print(e.__members__)

In [ ]:
from mlip_arena.models.utils import external_ase_calculator, EXTMLIPEnum

external_ase_calculator(EXTMLIPEnum.MACE)

In [ ]:
from mlip_arena.models.utils import MLIPEnum

for model in MLIPEnum:
    print(type(model.name), model.value)

In [ ]:
type(EXTMLIPEnum.MACE.value)

In [ ]:
import covalent as ct

from mace.calculators import mace_mp
from mlip_arena.tasks.diatomics import calculate_homonuclear_diatomics
from mlip_arena.models.utils import EXTMLIPEnum

local = ct.executor.LocalExecutor()

dispatch_id = ct.dispatch(calculate_homonuclear_diatomics)(calculator_name="MACE")

result = ct.get_result(dispatch_id)
print(result)